In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageDraw
from PIL import ImageColor
from PIL import ImageFont
import time
from scipy.stats import norm
from os import listdir
from os.path import isfile, join
import os

from numba import jit
import os.path
from skimage import io
from sklearn.utils.linear_assignment_ import linear_assignment
import glob
from filterpy.kalman import KalmanFilter
from tqdm import tnrange as trange
from tqdm import tqdm
%matplotlib inline
plt.style.use('ggplot')

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [0]:
!pip install filterpy==1.4.1

     |████████████████████████████████| 174kB 5.0MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.1-cp36-none-any.whl size=103795 sha256=5265a4ce94a1e64abd698077fff1f598b898f5513c18b1582bf3f593d93332f3
  Stored in directory: /root/.cache/pip/wheels/a5/9e/05/63c807a46727127f009bf08ce3079f2e9093226061131306c8
Successfully built filterpy


In [0]:
!wget https://motchallenge.net/data/MOT17Det.zip
!unzip MOT17Det.zip

--2020-01-13 19:26:15--  https://motchallenge.net/data/MOT17Det.zip
Resolving motchallenge.net (motchallenge.net)... 129.132.89.156
Connecting to motchallenge.net (motchallenge.net)|129.132.89.156|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1952547113 (1.8G) [application/zip]
Saving to: ‘MOT17Det.zip’

MOT17Det.zip        100%[===================>]   1.82G  18.5MB/s    in 1m 45s  

2020-01-13 19:28:02 (17.7 MB/s) - ‘MOT17Det.zip’ saved [1952547113/1952547113]

Archive:  MOT17Det.zip
  inflating: test/MOT17-01/seqinfo.ini  
  inflating: test/MOT17-03/seqinfo.ini  
  inflating: test/MOT17-06/seqinfo.ini  
  inflating: test/MOT17-07/seqinfo.ini  
  inflating: test/MOT17-08/seqinfo.ini  
  inflating: test/MOT17-12/seqinfo.ini  
  inflating: test/MOT17-14/seqinfo.ini  
  inflating: train/MOT17-02/seqinfo.ini  
  inflating: train/MOT17-04/seqinfo.ini  
  inflating: train/MOT17-05/seqinfo.ini  
  inflating: train/MOT17-09/seqinfo.ini  
  inflating: train/MOT17-10

In [0]:
#Download Pretrained weight Faster RCNN

In [0]:
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_nas_coco_2018_01_28.tar.gz
!tar -xvzf faster_rcnn_nas_coco_2018_01_28.tar.gz

--2020-01-13 19:29:03--  http://download.tensorflow.org/models/object_detection/faster_rcnn_nas_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.197.128, 2607:f8b0:400e:c08::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.197.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1173756273 (1.1G) [application/x-tar]
Saving to: ‘faster_rcnn_nas_coco_2018_01_28.tar.gz’

faster_rcnn_nas_coc 100%[===================>]   1.09G   135MB/s    in 9.1s    

2020-01-13 19:29:12 (123 MB/s) - ‘faster_rcnn_nas_coco_2018_01_28.tar.gz’ saved [1173756273/1173756273]

faster_rcnn_nas_coco_2018_01_28/
faster_rcnn_nas_coco_2018_01_28/model.ckpt.index
faster_rcnn_nas_coco_2018_01_28/checkpoint
faster_rcnn_nas_coco_2018_01_28/pipeline.config
faster_rcnn_nas_coco_2018_01_28/model.ckpt.data-00000-of-00001
faster_rcnn_nas_coco_2018_01_28/model.ckpt.meta
faster_rcnn_nas_coco_2018_01_28/saved_model/
faster_rcnn_nas_coco_

In [0]:
#Object Detection Uttilities

In [0]:
FASTER_RCNN_GRAPH_FILE = "faster_rcnn_nas_coco_2018_01_28/frozen_inference_graph.pb"
# only return boxes that meet a certain score threshold
def filter_boxes(min_score, boxes, scores, classes):
    n = len(classes)
    idxs = []
    for i in range(n):
        if scores[i] >= min_score and classes[i] == 1:
            idxs.append(i)
    
    filtered_boxes = boxes[idxs, ...]
    filtered_scores = scores[idxs, ...]
    filtered_classes = classes[idxs, ...]
    return filtered_boxes, filtered_scores, filtered_classes

  #[:, 0] means all rows and 0 column
def to_image_coords(boxes, height, width):
    box_coords = np.zeros_like(boxes)
    box_coords[:, 0] = boxes[:, 0] * height
    box_coords[:, 1] = boxes[:, 1] * width
    box_coords[:, 2] = boxes[:, 2] * height
    box_coords[:, 3] = boxes[:, 3] * width
    
    return box_coords

#draw boxes around region of interest
def draw_boxes(image, boxes, classes, thickness=4):
    """Draw bounding boxes on the image"""
    draw = ImageDraw.Draw(image)
    for i in range(len(boxes)):
        bot, left, top, right, idx = boxes[i, ...]
        class_id = int(classes[i])
        color = 'white'
 
        draw.line([(left, top), (left, bot), (right, bot), (right, top), (left, top)], width=thickness, fill=color)
        draw.text((left + 10, top + 10), "Human=" + str(idx),align="left")

#loading .pb file 
## Load a (frozen) Tensorflow model into memory.      
def load_graph(graph_file):
    """Loads a frozen inference graph"""
    graph = tf.Graph()
    with graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(graph_file, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return graph

In [0]:
#Object Tracking Utilities

In [0]:
@jit
def iou(bb_test,bb_gt):
  """
  Computes IUO between two bboxes in the form [x1,y1,x2,y2]
  """
  xx1 = np.maximum(bb_test[0], bb_gt[0])
  yy1 = np.maximum(bb_test[1], bb_gt[1])
  xx2 = np.minimum(bb_test[2], bb_gt[2])
  yy2 = np.minimum(bb_test[3], bb_gt[3])
  w = np.maximum(0., xx2 - xx1)
  h = np.maximum(0., yy2 - yy1)
  #wh is intersection are
  wh = w * h
  #intersection/union
  #union = area of gt + area of  prediction - intersection
  o = wh / ((bb_test[2]-bb_test[0])*(bb_test[3]-bb_test[1])
    + (bb_gt[2]-bb_gt[0])*(bb_gt[3]-bb_gt[1]) - wh)
  return(o)

def convert_bbox_to_z(bbox):
  """
  Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
  """
  w = bbox[2]-bbox[0]
  h = bbox[3]-bbox[1]
  x = bbox[0]+w/2.
  y = bbox[1]+h/2.
  s = w*h    #scale is just area
  r = w/float(h)
  return np.array([x,y,s,r]).reshape((4,1))

def convert_x_to_bbox(x,score=None):
  """
  Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
  """
  w = np.sqrt(x[2]*x[3])
  h = x[2]/w
  if(score==None):
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
  else:
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))

Linear Kalman Filter

In [0]:
class KalmanBoxTracker(object):
  """
  This class represents the internel state of individual tracked objects observed as bbox.
  """
  count = 0
  def __init__(self,bbox):
    """
    Initialises a tracker using initial bounding box.
    """
    #define constant velocity model
    self.kf = KalmanFilter(dim_x=7, dim_z=4)
    
    # Define the state transition matrix:
    self.kf.F = np.array([[1,0,0,0,1,0,0],
                          [0,1,0,0,0,1,0],
                          [0,0,1,0,0,0,1],
                          [0,0,0,1,0,0,0],  
                          [0,0,0,0,1,0,0],
                          [0,0,0,0,0,1,0],
                          [0,0,0,0,0,0,1]])
    
    self.kf.H = np.array([[1,0,0,0,0,0,0],
                          [0,1,0,0,0,0,0],
                          [0,0,1,0,0,0,0],
                          [0,0,0,1,0,0,0]])

    self.kf.R[2:,2:] *= 10.
    self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
    self.kf.P *= 10.
    self.kf.Q[-1,-1] *= 0.01
    self.kf.Q[4:,4:] *= 0.01

    self.kf.x[:4] = convert_bbox_to_z(bbox)
    self.time_since_update = 0
    self.id = KalmanBoxTracker.count
    KalmanBoxTracker.count += 1
    self.history = []
    self.hits = 0
    self.hit_streak = 0
    self.age = 0

  def update(self,bbox):
    """
    Updates the state vector with observed bbox.
    """
    self.time_since_update = 0
    self.history = []
    self.hits += 1
    self.hit_streak += 1
    self.kf.update(convert_bbox_to_z(bbox))

  def predict(self):
    """
    Advances the state vector and returns the predicted bounding box estimate.
    """
    if((self.kf.x[6]+self.kf.x[2])<=0):
      self.kf.x[6] *= 0.0
    self.kf.predict()
    self.age += 1
    if(self.time_since_update>0):
      self.hit_streak = 0
    self.time_since_update += 1
    self.history.append(convert_x_to_bbox(self.kf.x))
    return self.history[-1]

  def get_state(self):
    """
    Returns the current bounding box estimate.
    """
    return convert_x_to_bbox(self.kf.x)

Association and Assigment Utility

In [0]:
def associate_detections_to_trackers(detections,trackers,iou_threshold = 0.3):
  """
  Assigns detections to tracked object (both represented as bounding boxes)
  Returns 3 lists of matches, unmatched_detections and unmatched_trackers
  """
  if(len(trackers)==0):
    return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)
  iou_matrix = np.zeros((len(detections),len(trackers)),dtype=np.float32)

  for d,det in enumerate(detections):
    for t,trk in enumerate(trackers):
      iou_matrix[d,t] = iou(det,trk)
  matched_indices = linear_assignment(-iou_matrix)

  unmatched_detections = []
  for d,det in enumerate(detections):
    if(d not in matched_indices[:,0]):
      unmatched_detections.append(d)
  unmatched_trackers = []
  for t,trk in enumerate(trackers):
    if(t not in matched_indices[:,1]):
      unmatched_trackers.append(t)

  #filter out matched with low IOU
  matches = []
  for m in matched_indices:
    if(iou_matrix[m[0],m[1]]<iou_threshold):
      unmatched_detections.append(m[0])
      unmatched_trackers.append(m[1])
    else:
      matches.append(m.reshape(1,2))
  if(len(matches)==0):
    matches = np.empty((0,2),dtype=int)
  else:
    matches = np.concatenate(matches,axis=0)

  return matches, np.array(unmatched_detections), np.array(unmatched_trackers)


Simple Object Recognition and Tracking Example

In [0]:
class Sort(object):
  def __init__(self,max_age=1,min_hits=3):
    """
    Sets key parameters for SORT
    """
    self.max_age = max_age
    self.min_hits = min_hits
    self.trackers = []
    self.frame_count = 0

  def update(self,dets):
    """
    Params:
      dets - a numpy array of detections in the format [[x1,y1,x2,y2,score],[x1,y1,x2,y2,score],...]
    Requires: this method must be called once for each frame even with empty detections.
    Returns the a similar array, where the last column is the object ID.
    NOTE: The number of objects returned may differ from the number of detections provided.
    """
    self.frame_count += 1
    #get predicted locations from existing trackers.
    trks = np.zeros((len(self.trackers),5))
    to_del = []
    ret = []
    for t,trk in enumerate(trks):
      pos = self.trackers[t].predict()[0]
      trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]
      if(np.any(np.isnan(pos))):
        to_del.append(t)
    trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
    for t in reversed(to_del):
      self.trackers.pop(t)
    matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets,trks)

    #update matched trackers with assigned detections
    for t,trk in enumerate(self.trackers):
      if(t not in unmatched_trks):
        d = matched[np.where(matched[:,1]==t)[0],0]
        trk.update(dets[d,:][0])

    #create and initialise new trackers for unmatched detections
    for i in unmatched_dets:
        trk = KalmanBoxTracker(dets[i,:]) 
        self.trackers.append(trk)
    i = len(self.trackers)
    for trk in reversed(self.trackers):
        d = trk.get_state()[0]
        if((trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits)):
          ret.append(np.concatenate((d,[trk.id+1])).reshape(1,-1)) # +1 as MOT benchmark requires positive
        i -= 1
        #remove dead tracklet
        if(trk.time_since_update > self.max_age):
          self.trackers.pop(i)
    if(len(ret)>0):
      return np.concatenate(ret)
    return np.empty((0,5))

Main Loop

In [0]:
MYPATH = "train/MOT17-13/img1"

#Sort() for using SORT tracker
mot_tracker = Sort()

detection_graph = load_graph(FASTER_RCNN_GRAPH_FILE)
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
# Each box represents a part of the image where a particular object was detected.
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
# Each score represent how level of confidence for each of the objects.
# Score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')


for f in tqdm(sorted(listdir(MYPATH))):
  if isfile(join(MYPATH, f)):

    image = Image.open(os.path.join(MYPATH,f))

    #Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np = np.expand_dims(np.asarray(image, dtype=np.uint8), 0)

    with tf.Session(graph=detection_graph) as sess:    
         # Actual detection.            
        (boxes, scores, classes) = sess.run([detection_boxes, detection_scores, detection_classes], 
                                            feed_dict={image_tensor: image_np})
        
        # Visualization of the results of a detection.
        #REMOVE UNNECESSARY DIMENSIONS
        boxes = np.squeeze(boxes)
        scores = np.squeeze(scores)
        classes = np.squeeze(classes)

        confidence_cutoff = 0.8
         # Filter boxes with a confidence score less than `confidence_cutoff`
        boxes, scores, classes = filter_boxes(confidence_cutoff, boxes, scores, classes)
        
        # The current box coordinates are normalized to a range between 0 and 1.
    `   # This converts the coordinates actual location on the image.
        width, height = image.size
        box_coords = to_image_coords(boxes, height, width)

        #for sort tracker to update
        trackers = mot_tracker.update(box_coords)

        #trackers is a np array where each row contains a valid bounding box and track_id (last column)
        #print(trackers)

        #drwaing box around the tracked object
        # Each class with be represented by a differently colored box
        draw_boxes(image, trackers, classes)

        #saving output image frame with bounded boxes
        image.save(os.path.join('./result/',f)) 
        # plt.figure(figsize=(12, 8))
        # plt.imshow(image)


  0%|          | 1/750 [00:08<1:43:54,  8.32s/it]/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)

  3%|▎         | 19/750 [02:42<1:44:06,  8.54s/it]

KeyboardInterrupt: ignored

In [0]:
import shutil
shutil.make_archive('result1', 'zip', '/content/result')


'/content/result1.zip'